# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm

In [1]:
pip install pyowm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.4 MB/s eta 0:00:00


In [20]:
import pandas as pd
import requests as re
import pyowm
import json



In [41]:
with open('secrets.txt', 'r') as file:
    content = file.read()
    print(f"File content: {content}")
    if 'open-weather:' in content:
        api_key = content.split('open-weather:')[1].strip()
        print(f"Extracted API Key: {api_key}")

File content: fsq-token: fsq3cZO69vmEk1eghyZuQZcOZ2tymbgfywMhDglZ9OlA97k=
open-weather: 9ad8410a92ffffc960cfd653d4df2a21
Extracted API Key: 9ad8410a92ffffc960cfd653d4df2a21


## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [40]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = re.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 284.21,
  'feels_like': 283.72,
  'temp_min': 282.64,
  'temp_max': 285.01,
  'pressure': 1023,
  'humidity': 90,
  'sea_level': 1023,
  'grnd_level': 1019},
 'visibility': 10000,
 'wind': {'speed': 2.57, 'deg': 220},
 'clouds': {'all': 65},
 'dt': 1729548001,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1729492507,
  'sunset': 1729529683},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [42]:

from pyowm.owm import OWM
owm = OWM(api_key)
weather_mgr = owm.weather_manager()

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [38]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

params = {
    'appid': api_key,
    'units': 'metric'
}

response = re.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 11.03°C, Humidity: 90%, Wind Speed: 2.57 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [43]:
observation = weather_mgr.weather_at_place(city)

weather = observation.weather
temperature = weather.temperature('celsius')['temp']
humidity = weather.humidity
wind_speed = weather.wind()['speed']

print(f"City: {city}")
print(f"Temperature: {temperature}°C")
print(f"Humidity: {humidity}%")
print(f"Wind Speed: {wind_speed} m/s")

City: London
Temperature: 11.06°C
Humidity: 90%
Wind Speed: 2.57 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [34]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = re.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [35]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = re.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2024-10-22 00:00:00, Temp: 296.74°C
Time: 2024-10-22 03:00:00, Temp: 294.59°C
Time: 2024-10-22 06:00:00, Temp: 292.1°C
Time: 2024-10-22 09:00:00, Temp: 291.27°C
Time: 2024-10-22 12:00:00, Temp: 290.78°C
Time: 2024-10-22 15:00:00, Temp: 294.42°C
Time: 2024-10-22 18:00:00, Temp: 297.47°C
Time: 2024-10-22 21:00:00, Temp: 295.72°C
Time: 2024-10-23 00:00:00, Temp: 292.49°C
Time: 2024-10-23 03:00:00, Temp: 291.79°C
Time: 2024-10-23 06:00:00, Temp: 290.91°C
Time: 2024-10-23 09:00:00, Temp: 289.69°C
Time: 2024-10-23 12:00:00, Temp: 288.8°C
Time: 2024-10-23 15:00:00, Temp: 292.16°C
Time: 2024-10-23 18:00:00, Temp: 295.98°C
Time: 2024-10-23 21:00:00, Temp: 296.36°C
Time: 2024-10-24 00:00:00, Temp: 293.77°C
Time: 2024-10-24 03:00:00, Temp: 292.11°C
Time: 2024-10-24 06:00:00, Temp: 291.01°C
Time: 2024-10-24 09:00:00, Temp: 286.75°C
Time: 2024-10-24 12:00:00, Temp: 284.86°C
Time: 2024-10-24 15:00:00, Temp: 286.77°C
Time: 2024-10-24 18:00:00, Temp: 289.47°C
Time: 2024-10-24 21:00:00, Temp: 290

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [47]:
# I cannot proceed unfortunatelly, it seems that it needs a paid API.


daily_forecast = weather_mgr.forecast_at_place('Berlin,DE', 'daily').forecast
three_h_forecast = weather_mgr.forecast_at_place('Berlin,DE', '3h').forecast

UnauthorizedError: Invalid API Key provided

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [48]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = re.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 284.09°C, Fahrenheit: 543.362°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [50]:
weather = weather_mgr.weather_at_place('London,UK').weather
temp_dict_fahrenheit = weather.temperature('fahrenheit')
temp_dict_celsius = weather.temperature('celsius')

print(f"Temperature in Fahrenheit: {temp_dict_fahrenheit['temp']}°F")
print(f"Temperature in Celsius: {temp_dict_celsius['temp']}°C")

Temperature in Fahrenheit: 51.64°F
Temperature in Celsius: 10.91°C
